# Spark destributed mode
this notebook will show you the basic steps needed to be done in order to be able to use spark in distributed mode

We highly recommend that you work with jupyert store magic and place all sensative credentials in a different file.
the `%store -r` command below will retrive any variable being place in jupyterlab's cache.
in this example the `redshift_user` and `redshift_password` variables were declared in `mysecretstore.ipynb` and retrived by `%store -r` command.
see more information [here](https://connect.weworkers.io/pages/editpage.action?pageId=147130442)

%store -r

In [ ]:
import pyspark
import socket
import os
from pyspark.sql import SparkSession
from wework_utils.spark import getWek8sConf



All distributed mode configuration is hidden in the `getWek8sConf()`. 
if you want to see the explicit set of configuration please see [here](https://github.com/WeConnect/xprmt/blob/master/utils/wework_utils/spark.py).
All additional  configuratinon should be supplied by you (see below an example for declaring the number of executors)
For a complete list of spark configuration see [here](https://spark.apache.org/docs/latest/configuration.html#available-properties)

In [ ]:
session = SparkSession.builder.config(conf=getWek8sConf()) \
.config("spark.executor.instances","1").getOrCreate() \


Below is an example on how to work with saprk redshift connector. 
you should provide the redshift username/password and the table and the s3 uri in which data from redshift will be unload to.

In [ ]:
df = session.read \
    .format("com.databricks.spark.redshift") \
    .option("url", f"jdbc:redshift://redshift-production.weworkers.io:5439/analyticdb?user={redshift_user}&password={redshift_password}") \
    .option("dbtable", "mytable") \
    .option("tempdir", "s3a://mybacket/path/to/folder") \
    .option("tempformat","CSV") \
    .option("forward_spark_s3_credentials","true") \
    .load()
df.show()